In [2]:
# GPU availability
import tensorflow as tf
print('tf version {}'.format(tf.__version__))
print('Is GPU available {}'.format(tf.test.is_gpu_available()))
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))

tf version 1.13.1
Is GPU available False


In [3]:
# gym specifications
import gym
import time 
env_name = 'CartPole-v0'
env_name = 'Breakout-v0'
env_name = 'Pong-v0'
env = gym.make(env_name)
print(env_name)
print('observation dimensionality {}'.format(env.observation_space.shape ))
print('number of actions {}'.format(env.action_space.n))
print('meaning of actions {}'.format(env.unwrapped.get_action_meanings()))


Pong-v0
observation dimensionality (210, 160, 3)
number of actions 6
meaning of actions ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [7]:
# Train a model
%load_ext autoreload
%autoreload 2
import gym
import os
import matplotlib 
%matplotlib inline
from model import Model
from cfg import get_cfg
from utils import get_experiment_name
from utils import dotdict
import tensorflow as tf
tf.reset_default_graph()

# where to save experimental results
experiments_folder = os.path.join(os.getcwd(), 'results')
experiment_name, experiment_folder = get_experiment_name(experiments_folder)

# specify environment
env_name = 'four_rooms_maze'
env_name = 'BreakoutNoFrameskip-v4'
env_name = 'CartPole-v0'
env_name = 'PongNoFrameskip-v4'

# specify agent
agent_name = 'dqn'
agent_name = 'vpg'
agent_name = 'ppo'

# load default config parameters
cfg_env, cfg_agent = get_cfg(experiment_folder, env_name, agent_name)
cfg = cfg_env
cfg.update(cfg_agent)
cfg = dotdict(cfg)

# modify default config parameters
cfg.n_steps = 2000
cfg.batch_size = 1
cfg.log_step = 100
cfg.agent_buffer_start_size = 1000
cfg.verbose = False

# train model
model = Model(cfg)
model.train_model()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Tensor("baseline/dense/BiasAdd:0", shape=(?, 1), dtype=float32)
ep 3 st 100 ppo e.ret.tr -1.00 ep.ln 50 sp 0.0 ag.ls 0.36 bs.ls 0.65 
ep 5 st 200 ppo e.ret.tr -1.00 ep.ln 42 sp 144.1 ag.ls 0.45 bs.ls 0.69 
ep 8 st 300 ppo e.ret.tr -1.00 ep.ln 35 sp 124.6 ag.ls 0.54 bs.ls 0.72 
ep 11 st 400 ppo e.ret.tr -1.00 ep.ln 35 sp 124.6 ag.ls 0.54 bs.ls 0.70 
ep 14 st 500 ppo e.ret.tr -1.00 ep.ln 35 sp 125.5 ag.ls 0.53 bs.ls 0.70 
ep 17 st 600 ppo e.ret.tr -1.00 ep.ln 35 sp 126.9 ag.ls 0.53 bs.ls 0.70 
ep 18 st 700 ppo e.ret.tr -1.00 ep.ln 47 sp 147.1 ag.ls 0.53 bs.ls 0.64 
ep 19 st 800 ppo e.ret.tr -1.00 ep.ln 60 sp 122.9 ag.ls 0.48 bs.ls 0.58 
ep 21 st 900 ppo e.ret.tr -1.00 ep.ln 65 sp 72.4 ag.ls 0.38 bs.ls 0.55 
ep 22 st 1000 ppo e.ret.tr -1.00 ep.ln 77 sp 81.0 ag.ls 0.38 bs.ls 0.49 
ep 25 st 1100 ppo e.ret.tr -1.00 ep.ln 47 sp 62.6 ag.ls 0.43 bs.ls 0.62 
ep 28 st 1200 ppo e.ret.tr -1.00 ep.ln 35 sp 83.5 a

In [23]:
10000 * 2 * 84*84*4*4 / 1000 /1000

2257.92

In [1]:
# Load pretrained model
import os
from model import Model
import tensorflow as tf
tf.reset_default_graph()
experiment_folder = os.path.join(os.getcwd(), 'results/1')
model = Model.load(experiment_folder)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /Users/raoul/code/drl/results/1/ppo_ckpt-10
INFO:tensorflow:Restoring parameters from /Users/raoul/code/drl/results/1/baseline_ckpt-10


In [8]:
# Simulate one episode in OpenAI gym
import time 
obs, ext_return, done = model.env.reset(), 0., False
n_steps, _return = 0, 0
while not done:
    obs = model.encode_obs(obs)
    action = model.action(obs)
    obs, r, done, _ = model.env.step(action)
    print(action, r, done)
    _return += r
    n_steps += 1
    model.env.render()
    time.sleep(0.1)
model.env.close()
print(_return, n_steps)

3 1.0 False
0 1.0 False
1 1.0 False
3 1.0 False
0 1.0 False
2 1.0 False
2 1.0 False
2 1.0 False
1 1.0 False
2 1.0 False
0 1.0 False
2 1.0 False
3 1.0 False
2 1.0 False
1 1.0 False
2 1.0 False
0 1.0 False
1 1.0 False
2 1.0 False
0 1.0 False
0 1.0 False
0 1.0 False
2 1.0 False
2 1.0 False
2 1.0 False
1 1.0 False
2 1.0 False
2 1.0 False
0 1.0 False
0 1.0 False
0 1.0 False
1 1.0 False
0 1.0 False
1 1.0 False
2 1.0 False
1 1.0 False
1 1.0 False
2 1.0 False
1 1.0 False
1 1.0 False
2 1.0 False
2 1.0 False
1 0.0 True
42.0 43


In [20]:
# Simulate one episode in the Maze environment
import matplotlib 
%matplotlib nbagg
ext_return, n_steps, obs_list = model.simulate()
print(ext_return, n_steps, len(model.env.visited))
model.env.animate(obs_list)

AttributeError: 'AtariGame' object has no attribute 'visited'